In [1]:
import os
import re
import json
import zipfile
import requests
import numpy as np
import pandas as pd
from decimal import Decimal
# from tokens import YANDEX_ORG # Раскоменить для работы с API Яндекс.Организации и получения координат

In [2]:
def download(url)->str:
    response = requests.get(url) # Запрос к API Яндекс.Организации
    if response.status_code == 200:
        content_disposition = response.headers.get("Content-Disposition")
        if content_disposition:
            filename = re.findall("filename=(.+)", content_disposition)
            if filename:
                filename = filename[0]
                filename = re.sub(r'"', '', filename)
                with open(filename, "wb") as file:
                    file.write(response.content)
                return filename
            else:
                return "Не удалось получить имя файла"
        else:
            return "Не удалось получить информацию о загруженном файле"
    else:
        return "Ошибка при загрузке файла"

    
def unzip(filename:str)->str:
    filenames = []
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        filenames = zip_ref.namelist()
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extract(filenames[0])
    return filenames[0]

## Датасет с результатами ЕГЭ

[Ссылка на оригинальный датасет](https://data.mos.ru/odata/export/catalog?idFile=200291)



Набор данных «Результаты ЕГЭ» позволяет получить информацию об общих результатах государственной итоговой аттестации по программам среднего общего образования (далее – ГИА-11) каждой образовательной организации города Москвы.

В наборе представлены данные:

- об административных округах, районах, полных наименованиях образовательных организаций;

- о периоде, за который сдавался единый государственный экзамен (далее – ЕГЭ);

- о количестве участников, набравших не менее 200 баллов за 3 экзамена;

- о количестве участников, набравших более 160 баллов за 3 экзамена.

Результаты планируется публиковать ежегодно, таким образом можно будет проследить динамику повышения качества образования в Москве.


Современная ситуация в России

Согласно статье 59 Федерального закона от 29.12.2012 г. № 273-ФЗ «Об образовании в Российской Федерации», ГИА-11 завершает освоение основных образовательных программ среднего общего образования и является обязательной.

К ГИА-11 допускаются обучающиеся, не имеющие академической задолженности, в том числе за итоговое сочинение (изложение), и в полном объеме выполнившие учебный план или индивидуальный учебный план (имеющие годовые отметки по всем учебным предметам учебного плана за каждый год обучения по образовательной программе среднего общего образования не ниже удовлетворительных).

К ГИА-11 по учебным предметам, освоение которых завершилось ранее, допускаются обучающиеся X-XI (XII) классов, имеющие годовые отметки не ниже удовлетворительных по всем учебным предметам учебного плана за предпоследний год обучения.

Выпускники текущего года проходят ГИА-11 по двум обязательным учебным предметам (русский язык и математика) и учебным предметам по выбору.


Современная ситуация в Москве

Участники ГИА-11 могут узнать свои результаты по следующей схеме:

<table>
  <tr><td>обучающиеся образовательных организаций</td><td>в своей образовательной организации</td></tr>
  <tr><td>выпускники прошлых лет</td><td>в региональном центре обработки</td></tr>
</table>

На официальном Портале Мэра и Правительства Москвы www.mos.ru в разделе «Услуги и сервисы» можно оформить подписку на получение уведомлений о готовности результатов ГИА-11.

А знаете ли вы, что...

- Для проведения ГИА-11 предусмотрены следующие сроки: досрочный, основной и дополнительный периоды.

- Срок действия результатов ЕГЭ – 4 года, следующих за годом получения таких результатов.

- Результаты ЕГЭ каждого участника заносятся в федеральную информационную систему, бумажных свидетельств о результатах ЕГЭ не предусмотрено.


Материал подготовлен на основе информации data.mos.ru, dogm.mos.ru, mos.ru.

In [3]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=200291")
csvname = unzip(filename)

In [4]:
# датасет с результатами ЕГЭ, в нем нет адресов и нет координат, только названия школ
exams = pd.read_csv(csvname, sep=';')
exams = exams.drop(index=0) # удаляем первую строку с русским названием столбцов, английские ужесть в датасете
# удаляем пустой столбец по маске, т.к. его имя в часте нумерации меняется
exams = exams.drop(columns=[col for col in exams.columns if col.startswith("Unnamed:")])
# чистим имена школ от мусора, делаем все буквы маленькими, убираем кавычки, и др. найденные разночтения
exams['EDU_NAME'] = exams['EDU_NAME'].str.lower().str.strip().str.replace('«', '').str.replace('»', '') \
.str.replace('  ', ' ').str.replace('"', ' ').str.replace('общеобразовательн', 'образовательн').str.replace('№ ', '')
exams['geoData'] = None
exams.head()

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
1,8236,государственное бюджетное образовательное учре...,797946035,2016-2017,Восточный административный округ,район Богородское,5,19,None
2,8237,государственное казенное образовательное учреж...,797949150,2016-2017,Восточный административный округ,район Богородское,14,47,None
3,8238,государственное казенное образовательное учреж...,797949275,2016-2017,Восточный административный округ,район Вешняки,0,1,None
4,8239,частное учреждение образовательная организация...,797949425,2016-2017,Восточный административный округ,район Вешняки,0,6,None
5,8240,автономная некоммерческая организация средняя ...,797949570,2016-2017,Восточный административный округ,район Восточное Измайлово,1,5,None


In [5]:
exams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 1 to 4884
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                4884 non-null   object
 1   EDU_NAME          4884 non-null   object
 2   global_id         4884 non-null   object
 3   YEAR              4884 non-null   object
 4   AdmArea           4884 non-null   object
 5   District          4884 non-null   object
 6   PASSES_OVER_220   4884 non-null   object
 7   PASSER_UNDER_160  4884 non-null   object
 8   geoData           0 non-null      object
dtypes: object(9)
memory usage: 343.5+ KB


In [6]:
os.remove(filename)
os.remove(csvname)

## Датасет с координатами школ

[Датасет](https://data.mos.ru/odata/export/catalog?idFile=217731)

Набор данных содержит информацию об образовательных организациях города Москвы: адреса, контактные телефоны, местоположение на карте, адреса интернет-сайтов учреждений, а также информацию о доступности зданий для лиц с ограниченными возможностями.

В соответствии с Федеральным Законом от 29.12.2012 № 273-ФЗ «Об образовании в Российской Федерации» (Закон) изменилась классификация образовательных организаций. Ранее образовательные организации классифицировались по «типам» и «видам» в зависимости от возраста обучающихся, в настоящее время согласно п.2 ст.10 Закона образование подразделяется на общее, профессиональное, дополнительное и профессиональное обучение, обеспечивающие возможность реализации права на образование в течение всей жизни (непрерывное образование).

Современные московские образовательные организации, как правило, реализуют несколько уровней общего образования (п.4 ст.10 Закона) – дошкольное, начальное общее, основное общее, среднее общее. Они также могут предлагать среднее и высшее профессиональное образование (п.5 ст.10 Закона) и четкой привязки образовательных организаций к прежней классификации (Детский сад → Средняя школа → Учреждение среднего профессионального образования → Высшее учебное заведение) уже нет.

У больших школ появились новые возможности - образовательные организации получили ресурс для повышения доступности реализуемых ими образовательных программ для московских семей. В зависимости от предпочтений семей школа предлагает дополнительное образование: организует работу кружков, секций, курсов различной направленности для детей и взрослых в любом из своих зданий.

Именно поэтому местом предоставления той или иной образовательной услуги должны считаться все здания, входящие в состав образовательной организации. Места оказания услуг общего и профессионального образования, а также – место работы конкретного кружка вы можете уточнить у администрации или на официальном сайте образовательной организации.

In [7]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=217731")
csvname = unzip(filename)

In [8]:
# датасет со списком школ от департамента образования, в нем координаты, но есть не все школы, например нет частных
schools = pd.read_csv(csvname, sep=';')
schools = schools.drop(index=0)
schools = schools.drop(columns=[col for col in schools.columns if col.startswith("Unnamed:")])
schools['FullName'] = schools['FullName'].str.lower().str.strip().str.replace('«', '').str.replace('»', '') \
.str.replace('  ', ' ').str.replace('"', ' ').str.replace('общеобразовательн', 'образовательн').str.replace('№ ', '')
# Преобразование значения в колонке 'geoData' в список координат
# schools['geoData'] = schools['geoData'].str.replace('{coordinates=[', '').str.replace('], type=Polygon}', '')
schools.head()

,OrgType,ReorganizationStatus,IDEKIS,FullName,ShortName,Number,INN,KPP,OGRN,LegalOrganization,...,WebSite,EducationPrograms,EducationalServices,LicensingAndAccreditation,SupplementaryClassesAndGroups,NumberofStudentsInOO,TheContingentOfPreschoolersStudyingOO,global_id,geoData,geodata_center
1,прочее,действующее,14500,государственное автономное учреждение города м...,ГАУ Медиацентр,NaN,7718924940,771801001,1137746210161,автономное,...,shkolamoskva.ru,NaN,nested data,nested data,nested data,nested data,nested data,1139421628,"{coordinates=[[[37.704597369, 55.80046703], [3...",NaN
2,организация дополнительного профессионального ...,действующее,14494,государственное автономное образовательное учр...,ГАОУ ДПО «Корпоративный университет»,NaN,7714239823,771301001,1027700566761,автономное,...,corp-univer.ru,Профессиональное образование | высшее образова...,nested data,nested data,nested data,nested data,nested data,2386776971,"{coordinates=[[[[37.539544671, 55.799821354], ...",NaN
3,организация дополнительного профессионального ...,действующее,14506,государственное автономное образовательное учр...,ГАОУ ДПО МЦРПО,NaN,7707329480,772201001,1057707018192,автономное,...,mcrpo.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2645605456,"{coordinates=[[[[37.711001873, 55.762921204], ...",NaN
4,организация дополнительного профессионального ...,действующее,14605,государственное бюджетное образовательное учре...,ГБОУ ДПО МЦВП,NaN,7723029678,772301001,1027739293944,бюджетное,...,mcvp.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2645605458,"{coordinates=[[[[37.65201246, 55.742196123], [...",NaN
5,организация дополнительного профессионального ...,действующее,13070,государственное автономное образовательное учр...,ГАОУ ДПО ЦПМ,NaN,7725618950,770401001,1077761137222,автономное,...,cpm.dogm.mos.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2654965921,"{coordinates=[[[[37.56961301, 55.671061893], [...",NaN


In [9]:
schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 1 to 578
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   OrgType                                578 non-null    object
 1   ReorganizationStatus                   578 non-null    object
 2   IDEKIS                                 578 non-null    object
 3   FullName                               578 non-null    object
 4   ShortName                              578 non-null    object
 5   Number                                 475 non-null    object
 6   INN                                    578 non-null    object
 7   KPP                                    578 non-null    object
 8   OGRN                                   578 non-null    object
 9   LegalOrganization                      578 non-null    object
 10  Subordination                          578 non-null    object
 11  ChiefName          

In [10]:
os.remove(filename)
os.remove(csvname)

## Датасет с координатами загрязнений

[Ссылка на оригинальный датасет](https://data.mos.ru/odata/export/catalog?idFile=218714)

Набор данных позволяет получить информацию о месте отбора проб атмосферного воздуха, дате проведения отбора, основных результатах исследования. Для удобства ориентирования предусмотрена возможность просмотра местоположения контрольной точки исследования атмосферного воздуха на карте города Москвы.

 

В 2015 году Министерством природных ресурсов и экологии РФ был составлен экологический рейтинг российских городов, при составлении которого учитывались данные о состоянии атмосферного воздуха, водопользования, по выбросам от транспорта, об использовании городских территорий и энергопотреблении. По результатам рейтинга Москва - лидер по экологическому развитию среди остальных российских городов.

 

В городе Москве организована уникальная система мониторинга загрязнения атмосферного воздуха, в состав которой входят, кроме стационарных станций контроля загрязнения атмосферного воздуха, три передвижные лаборатории ГПБУ «Мосэкомониторинг».

 

Передвижная лаборатория – это автомобиль, оснащенный современным оборудованием, которое позволяет измерять загрязняющие вещества в режиме реального времени по основным загрязняющим веществам и отбирать пробы воздуха для последующего анализа в лаборатории на расширенный перечень веществ.

 

В Российской Федерации установлены гигиенические нормативы предельно допустимых концентраций (ПДК) загрязняющих веществ в атмосферном воздухе. В случае выявления в ходе рейдов передвижной лаборатории превышений нормативов информация о зафиксированных нарушениях направляется в соответствующие органы исполнительной власти для принятия мер.

 

А ЗНАЕТЕ ЛИ ВЫ, ЧТО?

 

Много сил, труда и терпения   потребовалось ученым, чтобы доказать, что воздух - не самостоятельное вещество, а сложная смесь газов. Впервые высказался о сложном составе воздуха ученый-художник Леонардо да Винчи (XV век).

Количественный состав воздуха  установил французский ученый Антуан Лоран Лавуазье. По результатам своих опытов он сделал вывод, что воздух в целом состоит из кислорода, пригодного для дыхания и горения, и азота в пропорциях 1/5 и 4/5 объема соответственно.

 

В настоящее время известно, что атмосферный воздух – это смесь газов, состоящая из постоянных и переменных компонентов. На высотах до 100 км относительный состав газов составляет: азот 78%, кислород 21%, около 1% инертного газа аргона, вклад остальных постоянных компонентов существенно меньше.

 

К переменным компонентам относится водяной пар, озон, двуокись углерода, диоксид серы, диоксид азота и другие, их доля составляет сотые и тысячные доли процентов и они называются «малыми» газами. Кроме этого, в атмосфере присутствуют взвешенные твердые частицы и аэрозоли.

 

Изучение загрязнения атмосферного воздуха началось с развитием промышленности и ростом городов.

 

В России одним из первых на вред промышленных выбросов в 1851 году обратил внимание Д.И.Менделеев в статье «О происхождении дыма», в которой писал: «Дым затемняет день, проникает в жилище, грязнит фасады зданий и общественные фабрики, причиняет многие неудобства и нездоровья».

 

Во второй половине XIX века в таком крупном индустриальном городе, как Чикаго началась борьба с дымом, в рамках которой появление густого дыма было объявлено нарушением общественного порядка, за которое причитался штраф, а с 1928 года в Чикаго существует сеть станций для наблюдения за загрязнением воздуха.

 

В СССР первые исследования по гигиене атмосферного воздуха проводились в 1929-1930 годах. Их целью было получить научно обоснованные материалы о степени и характере загрязнения атмосферного воздуха в связи с развитием и реконструкцией городов и строительством новых предприятий. Полученные данные использовались при разработке первых элементов санитарного законодательства о размещении промышленных предприятий и принципов организации санитарно-защитных зон между жилыми районами и предприятиями.

 

Постановление Совета Министров СССР от 29 мая 1949 г. «О мерах борьбы с загрязнением атмосферного воздуха и об улучшении санитарно-гигиенических условий населенных мест» положило начало проведению организованных мероприятий по охране атмосферного воздуха в масштабе всей страны, началось активное развитие законодательства об охране атмосферного воздуха.

 

На Конференции ООН по проблемам окружающей среды (Стокгольм, 1972 год) в рамках декларации был принят принцип о том, что природные ресурсы Земли, включая воздух, должны быть сохранены на благо нынешнего и будущих поколений путем тщательного планирования и управления по мере необходимости.

 

Стокгольмская конференция положила начало быстрому развитию национальных и международных организаций, направляющих и координирующих деятельность по сохранению окружающей среды. Если до Стокгольмской конференции государственные органы по охране окружающей среды были только в Швеции, США, Канаде, Великобритании и Японии, то после неё подобные структуры были организованы практически во всех государствах, в том числе в СССР.

 

В настоящее время на территории Российской Федерации действуют федеральные законы «Об охране окружающей среды» и «Об охране атмосферного воздуха», которые устанавливают правовые основы охраны атмосферного воздуха.

 

Материал подготовлен на основе информации: http://data.mos.ru/,  http://www.mosecom.ru/, http://www.meteorf.ru/, http://www.mnr.gov.ru/, http://www.ecoethics.ru/, http://energetika.in.ua/.


In [11]:
filename = download("https://data.mos.ru/odata/export/catalog?idFile=217731")
csvname = unzip(filename)

In [12]:
# датасет с указанием координаты загрязнений воздуха
ecology = pd.read_csv(csvname, sep=';')
ecology = ecology.drop(index=0)
ecology = ecology.drop(columns=[col for col in ecology.columns if col.startswith("Unnamed:")])
ecology.head()

,OrgType,ReorganizationStatus,IDEKIS,FullName,ShortName,Number,INN,KPP,OGRN,LegalOrganization,...,WebSite,EducationPrograms,EducationalServices,LicensingAndAccreditation,SupplementaryClassesAndGroups,NumberofStudentsInOO,TheContingentOfPreschoolersStudyingOO,global_id,geoData,geodata_center
1,прочее,действующее,14500,Государственное автономное учреждение города М...,ГАУ Медиацентр,NaN,7718924940,771801001,1137746210161,автономное,...,shkolamoskva.ru,NaN,nested data,nested data,nested data,nested data,nested data,1139421628,"{coordinates=[[[37.704597369, 55.80046703], [3...",NaN
2,организация дополнительного профессионального ...,действующее,14494,Государственное автономное образовательное учр...,ГАОУ ДПО «Корпоративный университет»,NaN,7714239823,771301001,1027700566761,автономное,...,corp-univer.ru,Профессиональное образование | высшее образова...,nested data,nested data,nested data,nested data,nested data,2386776971,"{coordinates=[[[[37.539544671, 55.799821354], ...",NaN
3,организация дополнительного профессионального ...,действующее,14506,Государственное автономное образовательное учр...,ГАОУ ДПО МЦРПО,NaN,7707329480,772201001,1057707018192,автономное,...,mcrpo.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2645605456,"{coordinates=[[[[37.711001873, 55.762921204], ...",NaN
4,организация дополнительного профессионального ...,действующее,14605,Государственное бюджетное образовательное учре...,ГБОУ ДПО МЦВП,NaN,7723029678,772301001,1027739293944,бюджетное,...,mcvp.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2645605458,"{coordinates=[[[[37.65201246, 55.742196123], [...",NaN
5,организация дополнительного профессионального ...,действующее,13070,Государственное автономное образовательное учр...,ГАОУ ДПО ЦПМ,NaN,7725618950,770401001,1077761137222,автономное,...,cpm.dogm.mos.ru,Дополнительное образование | дополнительное об...,nested data,nested data,nested data,nested data,nested data,2654965921,"{coordinates=[[[[37.56961301, 55.671061893], [...",NaN


In [13]:
os.remove(filename)
os.remove(csvname)

In [14]:
ecology.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 578 entries, 1 to 578
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   OrgType                                578 non-null    object
 1   ReorganizationStatus                   578 non-null    object
 2   IDEKIS                                 578 non-null    object
 3   FullName                               578 non-null    object
 4   ShortName                              578 non-null    object
 5   Number                                 475 non-null    object
 6   INN                                    578 non-null    object
 7   KPP                                    578 non-null    object
 8   OGRN                                   578 non-null    object
 9   LegalOrganization                      578 non-null    object
 10  Subordination                          578 non-null    object
 11  ChiefName          

## Координаты из списка школ в результаты ЭГЕ

In [15]:
exams['geoData'] = exams['EDU_NAME'].apply(lambda x: schools.loc[schools['FullName'] == x, 'geoData'].values[0] if x in schools['FullName'].values else None)
exams.head()

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
1,8236,государственное бюджетное образовательное учре...,797946035,2016-2017,Восточный административный округ,район Богородское,5,19,None
2,8237,государственное казенное образовательное учреж...,797949150,2016-2017,Восточный административный округ,район Богородское,14,47,"{coordinates=[[[[37.710083127, 55.825085846], ..."
3,8238,государственное казенное образовательное учреж...,797949275,2016-2017,Восточный административный округ,район Вешняки,0,1,"{coordinates=[[[[37.807984129, 55.726758653], ..."
4,8239,частное учреждение образовательная организация...,797949425,2016-2017,Восточный административный округ,район Вешняки,0,6,None
5,8240,автономная некоммерческая организация средняя ...,797949570,2016-2017,Восточный административный округ,район Восточное Измайлово,1,5,None


In [28]:
# Загрузка кэша из файла, если файл существует
YANDEX_ORG = 0 # Закоментить строку если есть ключ
cache_file = 'geo_cache.json'
if os.path.exists(cache_file):
    with open(cache_file, 'r') as f:
        geo_cache = json.load(f)
else:
    geo_cache = {}

# Функция для запроса к API Яндекса
def get_geo_data(row):
    if pd.notna(row['geoData']):
        return row['geoData']
    
    key = f"{row['AdmArea']}_{row['District']}_{row['EDU_NAME']}"
    if key in geo_cache:
        return geo_cache[key]
    
    url = "https://search-maps.yandex.ru/v1/"
    params = {
        "apikey": YANDEX_ORG,
        "lang": "ru_RU",
        "text": f"Москва, {row['AdmArea']}, {row['District']}, {row['EDU_NAME']}"
    }
    
    try:
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'features' in data and len(data['features']) > 0:
            geo_data = data['features'][0]['geometry']
            geo_cache[key] = geo_data
            return geo_data
        else:
            geo_cache[key] = None
            return None
    except Exception as e:
        print(f"Ошибка при запросе к API: {e}")
        geo_cache[key] = None
        return None

# Применение функции к строкам DataFrame
exams['geoData'] = exams.apply(get_geo_data, axis=1)

# Сохранение кэша в файл
with open(cache_file, 'w', encoding='utf-8') as f:
    json.dump(geo_cache, f, ensure_ascii=False, indent=4)

In [32]:
pd.set_option('display.max_colwidth', None)
exams[exams['geoData'].isna()]

,ID,EDU_NAME,global_id,YEAR,AdmArea,District,PASSES_OVER_220,PASSER_UNDER_160,geoData
574,8810,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,798000274,2016-2017,Юго-Восточный административный округ,Нижегородский район,5,22,None
1195,9432,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,880632637,2017-2018,Юго-Восточный административный округ,Нижегородский район,4,21,None
1525,9760,государственное бюджетное образовательное учреждение города москвы школа 439 интеллект,1011905093,2018-2019,Юго-Восточный административный округ,Нижегородский район,8,31,None


In [33]:
exams.to_csv('exams.csv', index=False, encoding='utf-8')